Dou et al. (2020) model

Agora em Julia versão 1.5.3

This script simulates the model

In [1]:
using LinearAlgebra, Statistics

# using Distributions, Expectations, NLsolve, Roots, Random, Plots, Parameters


using DataFrames
using Gadfly





Testando a solução do jogo

In [2]:
using NBInclude
@nbinclude("solve_model.ipynb")

#importou a função solve_tree

┌ Info: Precompiling NBInclude [0db19996-df87-5ea3-a455-e3a50d440464]
└ @ Base loading.jl:1278


solve_tree (generic function with 5 methods)

In [3]:
#Vh/D, L/D, Dj/D
data = [1.0, 0.25, 0.68]; 

#ρ, β, c0, λj
game_parameters = [0.884, 9.84, 0.044, 0.346];

In [4]:
s_W, j_W, Pst_array, Pjt_array, Ds, Dj = solve_tree(data, game_parameters);

  9.990974 seconds (36.24 M allocations: 4.522 GiB, 8.17% gc time)
 15.906430 seconds (37.51 M allocations: 7.913 GiB, 8.51% gc time)
 19.038876 seconds (41.72 M allocations: 10.702 GiB, 9.53% gc time)
 21.987984 seconds (44.98 M allocations: 13.054 GiB, 10.03% gc time)
 24.094338 seconds (47.56 M allocations: 14.973 GiB, 10.48% gc time)
 25.958872 seconds (49.48 M allocations: 16.466 GiB, 10.80% gc time)
 27.627828 seconds (51.28 M allocations: 17.897 GiB, 11.09% gc time)
 28.853620 seconds (52.51 M allocations: 18.904 GiB, 11.20% gc time)
 29.504661 seconds (53.44 M allocations: 19.679 GiB, 11.37% gc time)
 31.221634 seconds (54.32 M allocations: 20.419 GiB, 11.64% gc time)
 31.307498 seconds (54.96 M allocations: 20.966 GiB, 11.60% gc time)
 31.690341 seconds (55.60 M allocations: 21.502 GiB, 11.75% gc time)
 32.267691 seconds (56.52 M allocations: 22.359 GiB, 12.01% gc time)
329.667166 seconds (636.17 M allocations: 209.359 GiB, 10.93% gc time)


Simulation parameters

In [126]:
#θs0, θj0, μ
simulation_parameters = [28, 36, 4.566]

θs0, θj0, μ = simulation_parameters;
θs0 = Int64(θs0);
θj0 = Int64(θj0);

### Criando funções para simular o jogo

In [127]:
#parameters to some functions that are similar as the ones in solve_model

c0 = game_parameters[3];
c1 = Float64(0.015);

ρ = game_parameters[1];
λj = game_parameters[4];


Vmax = data[1];
L = data[2];
D = Ds + Dj;


In [128]:
#cost function
function Ct(t)
    #cost at period t=0(index1) is 0
    if(t <= 1)
        return 0
    else
        return c0 * D + c1 * (t-1) * D #test to make index==1 be t==0
    end
end



Ct (generic function with 1 method)

In [129]:
#liquidation payoffs

function s_L(t)
    return min(L - Ct(t), Ds)
end


function j_L(t)
    return min(L - Ct(t) - s_L(t), Dj)
end


j_L (generic function with 1 method)

In [130]:
function proposal(Pkt_array, t, hkt, lmt)
    
    #pkt array será sempre do propositor, quem responder às propostas apenas olhará o seu valor de continuação
    
    return policy, payoff_prop, payment, lm_next = Pkt_array[t, hkt, lmt, [end,end-1, 1, 2]]
    
end
    

proposal (generic function with 1 method)

In [131]:
#maximum value of reorganization each period
function Vt(Vmax, ρ, t)

    if(t <=1)
        return Vmax
    else
        #(t-2) instead of (t-1) because we shifted the indexes in the game so as to include t==0 at index==1
        return ρ^(t-2) * Vmax
    end


end

Vt (generic function with 1 method)

In [132]:
# answer_liq
function answer_liq(m_L, t, hm_next, payoff_prop)
    
    liq = (m_L(t), Vt(Vmax, ρ, t) * hm_next/100 - payoff_prop)
    
    payoff_liq, answer = findmax(liq)
    
    return payoff_liq, answer
end

answer_liq (generic function with 1 method)

In [133]:
function answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)
    
    reorg_value = (payment, Cont_val[t+1, hm_next, lm_next, lk_next])

    payoff_reorg, answer = findmax(reorg_value)
    
    return payoff_reorg, answer
end

answer_reorg (generic function with 1 method)

In [134]:
function update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
    
    if(propositor=="s")
        hst = hk_next
        lst = lk_next
        
        hjt = hm_next
        ljt = lm_next
        
    else
        hst = hm_next
        lst = lm_next
        
        hjt = hk_next
        ljt = lk_next
    end
    
    
    return hst, lst, hjt, ljt
end

        

update_beliefs (generic function with 1 method)

In [135]:
#código para tirar um draw da beta

function draw_beta(hkt, β=game_parameters[2], grid=100)

    u = rand()

    if(hkt == 100)
        
        return 100

    else
        
        x = 1.0 - exp(1.0/β * (log(1.0 - u) + β * log(1.0 - hkt/grid)))
        x = round(x * 100, digits=0)
#         return Int64(x * 100) #to convert in an integer
        return Int64(x)
        
    end
end

        

# @code_warntype draw_beta(10)



draw_beta (generic function with 3 methods)

In [136]:
function choose_parameters(propositor, hst, lst, hjt, ljt)
    
    if(propositor=="s")
        
        Pkt_array = Pst_array
        Cont_val = j_W
        prop_index = 1
        respondent_index = 2 #índice de j, para organizar o payoff
        m_L = j_L

        hkt = hst
        lkt = lst
        
        hmt = hjt
        lmt = ljt


        lk_next = hkt

        hk_next = draw_beta(hkt)
        hm_next = draw_beta(hmt)
        
    elseif(propositor=="j")
        Pkt_array = Pjt_array
        Cont_val = s_W
        prop_index = 2
        respondent_index = 1
        m_L = s_L

        hkt = hjt
        lkt = ljt
        
        hmt = hst
        lmt = lst


        lk_next = hkt


        hk_next = draw_beta(hkt)
        hm_next = draw_beta(hmt)

    else
        
        println("error: propositor not valid")
        
    end
    
    return Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next
end




#test
Hs0 = 30
Hj0 = 28


hst = Hs0
hjt = Hj0

#assumindo que lower bounds nos períodos iniciais são as próprias habilidades iniciais
lst = hst
ljt = hjt




choose_parameters("s", hst, lst, hjt, ljt);
    
    

In [178]:
function simulate_game(Hs0, Hj0, μ, λj, s_W=s_W, j_W=j_W, Pst_array=Pst_array, Pjt_array=Pjt_array, t=1, grid=100)
    
    hst = Hs0
    hjt = Hj0

    #assumindo que lower bounds nos períodos iniciais são as próprias habilidades iniciais
    lst = hst
    ljt = hjt
    
    result = zeros(5);
    
    #number of observed proposals
    observed_proposals = 0.0
    
    
    #recovering "T+1" from s_W 
    T = size(s_W,1)
    
    
    while(result[1]==0.0 && t < T)
        
        
        u = rand()

        if(u < λj)
            propositor = "j"

        else
            propositor = "s"

        end

        
        #setting the default variables according to the propositor
        Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next = choose_parameters(propositor, hst, lst, hjt, ljt)
        
        
        #proposal ####
        policy, payoff_prop, payment, lm_next = proposal(Pkt_array, t, hkt, lmt)

        #lm_next is the update of the adversary's lower bound
        if(lm_next==grid+2 || policy!= 3.0) #para evitar update de lowerbound quando proponente não propõe reorg
            lm_next = lmt
        else
            lm_next = Int64(lm_next)
        end
        
        
        
        if(policy==3.0)
            
            observed_proposals += 1.0
    
            payoff_respondent, answer = answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)


            if(answer==1.0)

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 3.0
                result[4] = t
                result[5] = observed_proposals

            else
                
                t+=1

                hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
            end

        elseif(policy==2.0)
            
            t+=1
            
            hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
            
            

        else      
            #(policy==1.0)
            
            observed_proposals += 1.0

            payoff_respondent, answer = answer_liq(m_L, t, hm_next, payoff_prop)

            if(answer==1.0)

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 1.0
                result[4] = t
                result[5] = observed_proposals

            else

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 3.0
                result[4] = t
                result[5] = observed_proposals


            end
        end


        if(t==T)
            
            result[1] = s_W[T, 1, 1, 1]#todos os valores finais de S são iguais, então acessarei o índice 1
            result[2] = j_W[T, 1, 1, 1]
            result[3] = 1.0
            result[4] = t
            result[5] = observed_proposals
            
        end
        




    end
    
    
    return result
end

        
        
#precisa colocar uma condição IF aqui para caso o jogo vá até o último período?
    

simulate_game (generic function with 7 methods)

In [192]:
simulate_game(θs0, θj0, μ, λj, s_W, j_W, Pst_array, Pjt_array)

5-element Array{Float64,1}:
 0.25
 0.14
 3.0
 1.0
 1.0

In [193]:
S = 40
N = 75
Results = zeros(S, N, 7)

#últimas entradas de Results são os valores das dívidas
Results[:,:, end-1] .= Dj
Results[:,:, end] .= Ds

@time begin
    

    for s in 1:S
        for n in 1:N
            Results[s,n,1:5] .= simulate_game(θs0, θj0, μ, λj)

        end
    end
    
end




  0.100413 seconds (487.53 k allocations: 22.627 MiB)


In [362]:
function loop_simulations(S, N, θs0, θj0, μ, λj, s_W=s_W, j_W=j_W, Pst_array=Pst_array, Pjt_array=Pjt_array)
    
    #S is the number of simulations
    #N is the number of observations
    
    #no futuro os argumentos podem ser arrays de arrays, assim ele faz o loop para cada cluster
    
    Results = zeros(S, N, 7)

    #últimas entradas de Results são os valores das dívidas
    Results[:,:, end-1] .= Dj
    Results[:,:, end] .= Ds


    for s in 1:S
        for n in 1:N
            Results[s,n,1:5] .= simulate_game(θs0, θj0, μ, λj)
        end
    end

    return Results
    
end

    
loop_simulations(40, 75, θs0, θj0, μ, λj);

In [366]:
function simulate_moments(S, N, θs0, θj0, μ, λj)
    
    
    Results = loop_simulations(S, N, θs0, θj0, μ, λj)
    
    
    Moments = zeros(S, 8)
    
    
    for s in 1:S
        
        DF = DataFrame()
        DF.payoff_s = Results[1,:,1];
        DF.payoff_j = Results[1,:,2];
        DF.out = Results[1,:,3];
        DF.t = Results[1,:,4];
        DF.observed_proposals = Results[1,:,5];
        DF.Dj = Results[1,:,6];
        DF.Ds = Results[1,:,7];
        
        
        DF.outcome = ifelse.(DF.out .==3.0, "R", "L");


        #desfazendo o deslocamento no índice de t
        DF.t = DF.t .- 1.0;

        # DF.incourt = ifelse.(DF.t .> 0.0, "incourt", "precourt");

        INCOURT = filter(DF -> DF.t .> 0.0, DF);
        PRECOURT = filter(DF -> DF.t .<= 0.0, DF);
        
        #cálculo dos momentos
        #1. avg log number of months between observed proposals incourt

        mm1 = INCOURT
        mm1 = log.(mm1.t .* μ ./ mm1.observed_proposals)
        mm1 = mean(mm1)

        #2. fraction reorganized given that the case went into court

        mm2 = INCOURT
        mm2 = size(filter(mm2 -> mm2.outcome .== "R", mm2),1)/ size(mm2,1)

        #3. ln duration of court cases in months

        mm3 = INCOURT
        mm3 = filter(mm3 -> mm3.t .> 0.0, mm3) #removendo os casos 0 para não poluir a média
        mm3.t = mm3.t .* μ
        mm3 = mean(log.(mm3.t)) #log here uses exp as base, so it's the same as ln

        #4. fraction of cases incourt
        mm4 = size(INCOURT,1) / size(DF, 1)

        #5. avg recovery rate for senior given precourt

        #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

        mm5 = PRECOURT
        mm5.payoff_s = mm5.payoff_s ./ PRECOURT.Ds
        mm5 = mean(mm5.payoff_s)


        #6. avg recovery rate for junior given precourt

        #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

        mm6 = PRECOURT
        mm6.payoff_j = mm6.payoff_j ./ PRECOURT.Dj
        mm6 = mean(mm6.payoff_j)


        #7. junior avg fraction gain given incourt

        mm7 = INCOURT
        mm7 = mean(mm7.payoff_j ./ (mm7.payoff_j .+ mm7.payoff_s )) 

        #8. total recovery rate given incourt

        mm8 = INCOURT
        mm8 = mean( mm8.payoff_s .+ mm8.payoff_j)
        
    
        
        Moments[s,:] .= [mm1, mm2, mm3, mm4, mm5, mm6, mm7, mm8]
        
    end
    
    return mean(Moments, dims=1)
end

        
        
#test
S = 40
N = 75

using BenchmarkTools
@btime simulate_moments(S, N, θs0, θj0, μ, λj)


    
    

  12.024 ms (139620 allocations: 5.06 MiB)


1×8 Array{Float64,2}:
 1.13276  1.0  2.23534  0.133333  0.712953  0.271853  0.46742  0.392606

In [338]:
#criei assim porque é um saco renomear colunas em Julia



DF = DataFrame()
DF.payoff_s = Results[1,:,1];
DF.payoff_j = Results[1,:,2];
DF.out = Results[1,:,3];
DF.t = Results[1,:,4];
DF.observed_proposals = Results[1,:,5];
DF.Dj = Results[1,:,6];
DF.Ds = Results[1,:,7];

In [339]:
DF

,payoff_s,payoff_j,out,t,observed_proposals,Dj,Ds
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.171052,0.175884,3.0,3.0,3.0,0.68,0.32
2,0.25,0.12,3.0,1.0,1.0,0.68,0.32
3,0.25,0.12,3.0,1.0,1.0,0.68,0.32
4,0.25,0.16,3.0,1.0,1.0,0.68,0.32
5,0.25,0.14,3.0,1.0,1.0,0.68,0.32
6,0.25,0.13,3.0,1.0,1.0,0.68,0.32
7,0.195362,0.206382,3.0,1.0,1.0,0.68,0.32
8,0.25,0.23,3.0,1.0,1.0,0.68,0.32
9,0.195362,0.206382,3.0,1.0,1.0,0.68,0.32


In [340]:
#como criar variável dummy
# Y = ifelse.(DF.x3 .== 3.0, "Reorganized", "Liquidated");

In [341]:
DF.outcome = ifelse.(DF.out .==3.0, "R", "L");


#desfazendo o deslocamento no índice de t
DF.t = DF.t .- 1.0;

# DF.incourt = ifelse.(DF.t .> 0.0, "incourt", "precourt");

INCOURT = filter(DF -> DF.t .> 0.0, DF);
PRECOURT = filter(DF -> DF.t .<= 0.0, DF);

Calculando momentos

In [344]:
#1. avg log number of months between observed proposals incourt

mm1 = INCOURT
mm1 = log.(mm1.t .* μ ./ mm1.observed_proposals)
mm1 = mean(mm1)

#2. fraction reorganized given that the case went into court

mm2 = INCOURT
mm2 = size(filter(mm2 -> mm2.outcome .== "R", mm2),1)/ size(mm2,1)

#3. ln duration of court cases in months

mm3 = INCOURT
mm3 = filter(mm3 -> mm3.t .> 0.0, mm3) #removendo os casos 0 para não poluir a média
mm3.t = mm3.t .* μ
mm3 = mean(log.(mm3.t)) #log here uses exp as base, so it's the same as ln

#4. fraction of cases incourt
mm4 = size(INCOURT,1) / size(DF, 1)

#5. avg recovery rate for senior given precourt

#aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

mm5 = PRECOURT
mm5.payoff_s = mm5.payoff_s ./ PRECOURT.Ds
mm5 = mean(mm5.payoff_s)


#6. avg recovery rate for junior given precourt

#aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

mm6 = PRECOURT
mm6.payoff_j = mm6.payoff_j ./ PRECOURT.Dj
mm6 = mean(mm6.payoff_j)


#7. junior avg fraction gain given incourt

mm7 = INCOURT
mm7 = mean(mm7.payoff_j ./ (mm7.payoff_j .+ mm7.payoff_s )) 

#8. total recovery rate given incourt

mm8 = INCOURT
mm8 = mean( mm8.payoff_s .+ mm8.payoff_j)

1.1584555061134787